1- import libraries

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns



2-Load Dataset

In [4]:
img_size = (224, 224)
batch_size = 16

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "data/processed/train",
    image_size=img_size,
    batch_size=batch_size,
    seed=42
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "data/processed/val",
    image_size=img_size,
    batch_size=batch_size,
    seed=42
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "data/processed/test",
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
print(f"Number of classes: {len(class_names)}")
print(f"Class names: {class_names}")

train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)


Found 507 files belonging to 9 classes.
Found 142 files belonging to 9 classes.
Found 82 files belonging to 9 classes.
Number of classes: 9
Class names: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


3-Load EfficientNetB3 Model

In [6]:
efficientnet_model = keras.models.load_model(
    "Models/EfficientNetB3/efficientnetb3_model.keras"
)

print("EfficientNetB3 loaded successfully!")


EfficientNetB3 loaded successfully!


4-Load ResNet50 Model

In [7]:
resnet_model = keras.models.load_model("resnet50_final.h5")
print("ResNet50 loaded successfully!")


ResNet50 loaded successfully!


5-Load MobileNet Model

In [8]:
mobilenet_model = keras.models.load_model(
    "Models/MobileNet/mobilenet_model.keras"
)

print("MobileNet loaded successfully!")

MobileNet loaded successfully!
